<a href="https://colab.research.google.com/github/dominikmeyer95/academic-output/blob/feature%2Fadvanced_derivatives/advanced_derivatives/retail_options_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Derivatives

* Group: Dominik Meyer, Sean Goedgeluk, Krzysztof Rentflejsz
* Assignment: Exploit retail investors' impact on option pricing 

# Key Ressources & Notes

**Tebaldi's opener:**
* Retail investors do not use options for hedging but for leveraging upside
* Thus, retail investors put price pressure on the call side of option markets
* This phenomenon becomes more severe the easier the access to derivatives becomes, e.g. via neobroker like Robinhood

**Notes on online articles:**
* https://qz.com/2043668/the-real-reason-options-trading-pushed-robinhood-shares-higher
* https://www.cboe.com/insights/posts/how-meme-stocks-impact-options-trading/

* In meme stocks, delta hedging does not work anymore, because the typical negative correlation of the assets in the hedging portfolio (options and the underlying stock) has turned. When a stock price goes up, its implied volatility typically goes down as if a stock is going up, in theory it’s because investors have more certainty about its future cashflows and profits. Meme stocks flip this around: The equity may shoot up in price, but it’s reasonable to expect that this price increase is highly unstable, thus the implied volatility stays elevated. Investment banks hence hedge a short call buy a long call positionas this hedges the volatility risk as well.

**Ideas:**
* Have a look at implied volatility and its variation
* Have a look at the volatility skew as it might be less skewed based on that information
* (Have a look at the delta Skew, which indicates the demand for downside versus upside potential in options)

# Table of Contents

In [1]:
# import required packages
import numpy as np
import pandas as pd
import scipy as sp
import plotly.express as px
import plotly.graph_objects as go

In [13]:
# import wrds data
wrds_data = pd.read_csv('https://raw.githubusercontent.com/dominikmeyer95/academic-output/feature/advanced_derivatives/advanced_derivatives/wrds_data.csv')
wrds_data

,secid,date,days,forward_price,strike_price,premium,impl_volatility,delta,gamma,theta,vega,cp_flag,ticker,index_flag
0,108105,20000103,10,1457.239519,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,C,SPX,1
1,108105,20000103,30,1461.336754,1461.336754,34.225470,0.205810,0.511405,0.004641,-244.081939,166.247836,C,SPX,1
2,108105,20000103,60,1467.704629,1467.704629,49.951510,0.212584,0.516454,0.003173,-187.871518,234.827412,C,SPX,1
3,108105,20000103,91,1474.440355,1474.440355,61.852062,0.213935,0.520178,0.002557,-159.976593,288.841452,C,SPX,1
4,108105,20000103,122,1481.293004,1481.293004,70.534737,0.210888,0.522796,0.002238,-141.446275,334.051190,C,SPX,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227315,143439,20211231,182,1059.876441,1059.876441,179.488981,0.606810,-0.415852,0.000863,-173.793997,291.030314,P,TSLA,0
227316,143439,20211231,273,1061.801703,1061.801703,219.645565,0.608318,-0.397238,0.000696,-139.698999,352.374750,P,TSLA,0
227317,143439,20211231,365,1064.104282,1064.104282,253.801860,0.609903,-0.381547,0.000594,-118.857932,402.729287,P,TSLA,0
227318,143439,20211231,547,1069.958000,1069.958000,311.663307,0.615708,-0.355398,0.000471,-94.108183,481.428596,P,TSLA,0
